In [1]:
import pandas as pd
import numpy as np

In [86]:
# open key_rows_with_tracts.xlsx
krwt = pd.read_excel('key_rows_with_tracts.xlsx')
# drop geo_filename and tract_filename columns
krwt = krwt.drop(columns=['geo_filename', 'tract_filename'])
krwt

,filename,city,state,geography,district,office,year,tracts,percents
0,Berkeley_11032020_MayorBerkeleyRCV.csv,Berkeley,CA,city,NaN,Mayor,2020,NaN,NaN
1,Berkeley_11062012_Mayor.csv,Berkeley,CA,city,NaN,Mayor,2012,NaN,NaN
2,Berkeley_11082016_Mayor.csv,Berkeley,CA,city,NaN,Mayor,2016,NaN,NaN
3,Berkeley_03072017_CityCouncilDistrict4.csv,Berkeley,CA,city council,4.0,City council,2017,"[422400, 422300, 421700, 423500, 422800, 42360...","[0.8959146638358644, 0.007147169403565304, 0.0..."
4,Berkeley_11022010_CityCouncilDistrict1.csv,Berkeley,CA,city council,1.0,City council,2010,"[421800, 420400, 422100, 422200, 422000, 42050...","[0.2897100982447431, 0.001545630906237761, 0.9..."
...,...,...,...,...,...,...,...,...,...
170,SanLeandro_11062018_CountyCouncilDistrict3.csv,San Leandro,CA,city council,3.0,County board of supervisors,2018,"[433400, 432501, 433104, 432400, 433200, 43250...","[0.1405611517171908, 0.637280465424566, 0.0151..."
171,SanLeandro_11062018_CountyCouncilDistrict5.csv,San Leandro,CA,city council,5.0,County board of supervisors,2018,"[410400, 410200, 410000, 432700, 409200, 41010...","[0.0003256444881375276, 4.247353688978054e-05,..."
172,SanLeandro_11082016_CountyCouncilDistrict2.csv,San Leandro,CA,city council,2.0,County board of supervisors,2016,"[433700, 433800, 433104, 433200, 432700, 43300...","[0.005588870087866759, 0.10596707456802083, 0...."
173,SanLeandro_11032020_MemberCityCouncilDist2SanL...,San Leandro,CA,city council,2.0,City council,2020,"[432800, 433700, 433200, 433000, 433102, 43360...","[6.159201370597147e-05, 0.0035482507016996586,..."


In [23]:
# open fips.csv
fips = pd.read_csv('fips.csv')
fips

,city,county,state,state fips,county fips,place
0,berkeley,alameda,california,6,1,6000
1,san leandro,alameda,california,6,1,68084
2,san francisco,san francisco,california,6,75,67000
3,oakland,alameda,california,6,1,53000
4,minneapolis,hennepin,minnesota,27,53,43000


In [7]:
abbreviation_to_name = {
    # https://en.wikipedia.org/wiki/List_of_states_and_territories_of_the_United_States#States.
    "AK": "Alaska",
    "AL": "Alabama",
    "AR": "Arkansas",
    "AZ": "Arizona",
    "CA": "California",
    "CO": "Colorado",
    "CT": "Connecticut",
    "DE": "Delaware",
    "FL": "Florida",
    "GA": "Georgia",
    "HI": "Hawaii",
    "IA": "Iowa",
    "ID": "Idaho",
    "IL": "Illinois",
    "IN": "Indiana",
    "KS": "Kansas",
    "KY": "Kentucky",
    "LA": "Louisiana",
    "MA": "Massachusetts",
    "MD": "Maryland",
    "ME": "Maine",
    "MI": "Michigan",
    "MN": "Minnesota",
    "MO": "Missouri",
    "MS": "Mississippi",
    "MT": "Montana",
    "NC": "North Carolina",
    "ND": "North Dakota",
    "NE": "Nebraska",
    "NH": "New Hampshire",
    "NJ": "New Jersey",
    "NM": "New Mexico",
    "NV": "Nevada",
    "NY": "New York",
    "OH": "Ohio",
    "OK": "Oklahoma",
    "OR": "Oregon",
    "PA": "Pennsylvania",
    "RI": "Rhode Island",
    "SC": "South Carolina",
    "SD": "South Dakota",
    "TN": "Tennessee",
    "TX": "Texas",
    "UT": "Utah",
    "VA": "Virginia",
    "VT": "Vermont",
    "WA": "Washington",
    "WI": "Wisconsin",
    "WV": "West Virginia",
    "WY": "Wyoming",
    # https://en.wikipedia.org/wiki/List_of_states_and_territories_of_the_United_States#Federal_district.
    "DC": "District of Columbia",
    # https://en.wikipedia.org/wiki/List_of_states_and_territories_of_the_United_States#Inhabited_territories.
    "AS": "American Samoa",
    "GU": "Guam GU",
    "MP": "Northern Mariana Islands",
    "PR": "Puerto Rico PR",
    "VI": "U.S. Virgin Islands",
}

In [71]:
# open acs data\place_data.csv
place_data = pd.read_csv('acs data/place_data.csv')

place_data

,city_name,place_code,county_name,county_code,state_name,state_code,year,DP02_0119EA,DP03_0028EA,DP02_0073EA,...,DP02_0154EA,DP02_0025EA,DP02_0067E,DP02_0100E,DP03_0036E,DP02_0106EA,DP05_0041EA,DP02_0102E,DP02_0053E,DP03_0101EA
0,berkeley,6000,alameda,1,california,6,2020,NaN,NaN,NaN,...,NaN,NaN,74177.0,858,608,NaN,NaN,26086,45047,NaN
1,berkeley,6000,alameda,1,california,6,2012,NaN,NaN,NaN,...,NaN,NaN,-888888888.0,24070,673,NaN,NaN,19292,1549,NaN
2,berkeley,6000,alameda,1,california,6,2016,NaN,NaN,NaN,...,NaN,NaN,54772.0,24889,798,NaN,NaN,15482,1703,NaN
3,minneapolis,43000,hennepin,53,minnesota,27,2021,NaN,NaN,NaN,...,NaN,NaN,260985.0,2198,4945,NaN,NaN,61781,112994,NaN
4,minneapolis,43000,hennepin,53,minnesota,27,2013,NaN,NaN,NaN,...,NaN,NaN,-888888888.0,61918,4715,NaN,NaN,51634,6909,NaN
5,minneapolis,43000,hennepin,53,minnesota,27,2009,NaN,NaN,(X),...,NaN,NaN,-888888888.0,56744,4909,NaN,NaN,28742,5899,(X)
6,minneapolis,43000,hennepin,53,minnesota,27,2017,NaN,NaN,NaN,...,NaN,NaN,252278.0,3951,4899,NaN,NaN,19543,114446,NaN
7,oakland,53000,alameda,1,california,6,2010,NaN,NaN,NaN,...,NaN,NaN,-888888888.0,108109,4031,NaN,NaN,106337,6979,NaN
8,oakland,53000,alameda,1,california,6,2020,NaN,NaN,NaN,...,NaN,NaN,276477.0,2345,4033,NaN,NaN,115087,94803,NaN
9,oakland,53000,alameda,1,california,6,2014,NaN,NaN,NaN,...,NaN,NaN,-888888888.0,112305,4482,NaN,NaN,99950,6977,NaN


In [105]:
all_data = []
for index, row in krwt.iterrows():
    city_name = row['city'].lower()
    year = row['year']

    tracts = str(row['tracts']).strip('][').split(', ')
    weights = str(row['percents']).strip('][').split(', ')
    
    election_data = {}

    if row['geography'] == 'city':
        acs_data = pd.read_csv(f'acs data\place_data.csv')
        # find row where city_name = city_name and year = year
        city_row = acs_data.loc[(acs_data['city_name'] == city_name) & (acs_data['year'] == year)]
        # for each column in the row that starts with DP, take its value, multiply it by weight, and add to data with column as key and value as value
        for column in city_row.columns:
            if column.startswith('DP'):
                value = city_row[column].values[0]
                # if value cannot be converted to number or is NaN, continue, otherwise convert to int or float
                try:
                    value = int(value)
                except:
                    try:
                        value = float(value)
                    except:
                        continue
                # if value not nan
                if not np.isnan(value):
                    election_data[column] = election_data.get(column, 0) + value
    else:
        # open acs data\{city_name}\tract_data.csv
        acs_data = pd.read_csv(f'acs data\{city_name}_tract_data.csv')

        for i in range(len(tracts)):
            tract = tracts[i]
            weight = weights[i]

            acs_data = pd.read_csv(f'acs data\{city_name}_tract_data.csv')

            # get row in acs_data where city_name is city_name and tract is tract
            tract_row = acs_data.loc[(acs_data['city_name'] == city_name) & (acs_data['tract'] == int(tract)) & (acs_data['year'] == year)]
            
            # for each column in the row that starts with DP, take its value, multiply it by weight, and add to data with column as key and value as value
            for column in tract_row.columns:
                if column.startswith('DP'):
                    value = tract_row[column].values[0]
                    # if value cannot be converted to number or is NaN, continue, otherwise convert to int or float
                    try:
                        value = int(value)
                    except:
                        try:
                            value = float(value)
                        except:
                            continue
                    # if value not nan
                    if not np.isnan(value):
                        election_data[column] = election_data.get(column, 0) + value * float(weight)
    all_data.append(election_data)

C:\Users\User\AppData\Local\Temp\ipykernel_30796\1295154982.py:32: DtypeWarning: Columns (36,42,44,75,95,101,116,125,131,150,174,182,186,199,210,214,225,238,258,264,288,295,330,345,350,375,378,383,390,402,410,414,416,428,455,458,480,483,493,503,514,526,529,531,544,546,552,562,573,578,601,641,644,649,665,685,704,713,723,724,739,755) have mixed types. Specify dtype option on import or set low_memory=False.
  acs_data = pd.read_csv(f'acs data\{city_name}_tract_data.csv')
C:\Users\User\AppData\Local\Temp\ipykernel_30796\1295154982.py:38: DtypeWarning: Columns (36,42,44,75,95,101,116,125,131,150,174,182,186,199,210,214,225,238,258,264,288,295,330,345,350,375,378,383,390,402,410,414,416,428,455,458,480,483,493,503,514,526,529,531,544,546,552,562,573,578,601,641,644,649,665,685,704,713,723,724,739,755) have mixed types. Specify dtype option on import or set low_memory=False.
  acs_data = pd.read_csv(f'acs data\{city_name}_tract_data.csv')
C:\Users\User\AppData\Local\Temp\ipykernel_30796\12951

In [110]:
krwt_df = krwt.copy()
all_data_df = pd.DataFrame(all_data)
result_df = pd.concat([krwt_df, all_data_df], axis=1)

In [112]:
#save to csv
result_df.to_csv('final_data.csv', index=False)
